### Tecnolgías del Lenguaje. Entregable 4
---
#### Generación de embeddings de referencia para los rasgos Big Five por rasgo (NEO-FFI)

En este paso se generan los vectores de referencia que representan cada uno de los cinco grandes rasgos de personalidad (Big Five) según el cuestionario NEO-FFI. Este cuestionario contiene afirmaciones que describen comportamientos, actitudes y emociones típicas asociadas a cada rasgo, tanto en su polaridad positiva (alta puntuación) como negativa (baja puntuación).

Cada ítem del cuestionario se transforma en un embedding semántico utilizando el modelo `all-MiniLM-L6-v2` de Sentence Transformers (es un SBERT refinado), que produce representaciones vectoriales de frases basadas en su significado. Los embeddings correspondientes a las frases de un mismo rasgo se promedian para obtener un vector medio que sintetiza el contenido semántico característico de ese rasgo. Estos vectores serán muy útiles, ya que captan el significado global de todas las frases que apuntan hacia el polo positivo o negativo de un rasgo y lo representan de forma única, reduciendo drásticamente los tiempos de ejecución.

El resultado es un conjunto de cinco vectores, uno por rasgo que actúan como puntos de referencia en el espacio semántico.
**Estos vectores servirán posteriormente para calcular la similitud entre los textos publicados por los usuarios y las descripciones prototípicas de cada rasgo**, permitiendo así estimar sus puntuaciones en el modelo de personalidad Big Five.

El archivo resultante (`material/trait_embeddings.csv`) contiene los embeddings medios **para cada rasgo**, y será utilizado en los pasos siguientes del análisis.

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

# ------------------------------------------------------
# 1️⃣ Definir los ítems del cuestionario NEO-FFI
# ------------------------------------------------------
neo_ffi = {
    "Agreeableness": {
        "positive": [
            "I try to be courteous to everyone I meet.",
            "I would rather cooperate with others than compete with them.",
            "Most people I know like me.",
            "I generally try to be thoughtful and considerate."
        ],
        "negative": [
            "I often get into arguments with my family and co-workers.",
            "Some people think I’m selfish and egotistical.",
            "I tend to be cynical and skeptical of other’s intentions.",
            "I believe that most people will take advantage of you if you let them.",
            "Some people think of me as cold and calculating.",
            "I’m hard-headed and tough-minded in my attitudes.",
            "If I don’t like people, I let them know it.",
            "If necessary, I am willing to manipulate people."
        ]
    },
    "Openness": {
        "positive": [
            "I am intrigued by the patterns I find in art and nature.",
            "I often try new and foreign foods.",
            "Sometimes when I am reading poetry or looking at a work of art, I feel a chill or wave of excitement.",
            "I have a lot of intellectual curiosity.",
            "I often enjoy playing with theories or abstract ideas."
        ],
        "negative": [
            "I don’t like to waste my time daydreaming.",
            "Once I find the right way to do something, I stick to it.",
            "I believe letting students hear controversial speakers can only confuse and mislead them.",
            "Poetry has little or no effect on me.",
            "I seldom notice the moods or feelings that different environments produce.",
            "I believe we should look to our religious authorities for decisions on moral issues.",
            "I have little interest in speculating on the nature of the universe or the human condition."
        ]
    },
    "Conscientiousness": {
        "positive": [
            "I keep my belongings clean and neat.",
            "I’m pretty good about pacing myself so as to get things done on time.",
            "I try to perform all the tasks assigned to me conscientiously.",
            "I have a clear set of goals and work toward them in an orderly fashion.",
            "I work hard to accomplish my goals.",
            "When I make a commitment, I can always be counted on to follow through.",
            "I am a productive person who always gets the job done.",
            "I strive for excellence in everything I do."
        ],
        "negative": [
            "I am not a very methodical person.",
            "I waste a lot of time before settling down to work.",
            "Sometimes I’m not as dependable or reliable as I should be.",
            "I never seem to be able to get organized."
        ]
    },
    "Extraversion": {
        "positive": [
            "I like to have a lot of people around me.",
            "I laugh easily.",
            "I really enjoy talking to people.",
            "I like to be where the action is.",
            "I often feel as if I am bursting with energy.",
            "I am a cheerful, high-spirited person.",
            "My life is fast-paced.",
            "I am a very active person."
        ],
        "negative": [
            "I don’t consider myself especially ‘lighthearted’.",
            "I usually prefer to do things alone.",
            "I am not a cheerful optimist.",
            "I would rather go my own way than be a leader of others."
        ]
    },
    "Neuroticism": {
        "positive": [
            "I often feel inferior to others.",
            "When I’m under a great deal of stress, sometimes I feel like I’m going to pieces.",
            "I often feel tense and jittery.",
            "Sometimes I feel completely worthless.",
            "I often get angry at the way people treat me.",
            "Too often, when things go wrong, I get discouraged and feel like giving up.",
            "I often feel helpless and want someone else to solve my problems.",
            "At times I have been so ashamed I just wanted to hide."
        ],
        "negative": [
            "I am not a worrier.",
            "I rarely feel lonely or blue.",
            "I rarely feel fearful or anxious.",
            "I am seldom sad or depressed."
        ]
    }
}

# ------------------------------------------------------
# 2️⃣ Cargar modelo
# ------------------------------------------------------
print("🔹 Loading model...")
model = SentenceTransformer("all-MiniLM-L6-v2")

# ------------------------------------------------------
# 3️⃣ Calcular un embedding único por rasgo
# ------------------------------------------------------
results = []
for trait, polarities in neo_ffi.items():
    print(f"→ Encoding {trait}...")
    
    # Unir frases positivas y negativas
    all_sentences = polarities["positive"] + polarities["negative"]
    
    emb = model.encode(all_sentences, normalize_embeddings=True)
    mean_emb = np.mean(emb, axis=0)
    
    results.append({
        "trait": trait,
        "embedding": mean_emb
    })

# ------------------------------------------------------
# 4️⃣ Guardar como CSV
# ------------------------------------------------------
emb_dim = results[0]["embedding"].shape[0]
emb_cols = [f"dim_{i}" for i in range(emb_dim)]

df = pd.DataFrame([
    {"trait": r["trait"], **{f"dim_{i}": r["embedding"][i] for i in range(emb_dim)}}
    for r in results
])

df.to_csv("material/trait_embeddings.csv", index=False)
print(f"✅ Saved trait_embeddings.csv with shape {df.shape}")

C:\Users\Usuario\anaconda3\envs\teclin\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


🔹 Loading model...


C:\Users\Usuario\anaconda3\envs\teclin\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


→ Encoding Agreeableness...
→ Encoding Openness...
→ Encoding Conscientiousness...
→ Encoding Extraversion...
→ Encoding Neuroticism...
✅ Saved trait_embeddings.csv with shape (5, 385)
